In [1]:
import facebook
import json
import requests
import random

token = 'ACCESS_TOKEN'
graph = facebook.GraphAPI(access_token = token, version = '2.7')
fetch_count = 2000 #每個專頁抓2000篇文章
test_count = 100   #隨機取樣其中100筆做測試資料

# 柯文哲、朱立倫、鄭文燦、林佳龍、賴清德、陳菊　粉絲專頁ID
page_ids = ['136845026417486', '10150145806225128', '333058400178329', '153819538009272', '152472278103133', '232716627404']

# Fetch Pages Posts
posts_text = []
posts_label = []

for page_id in page_ids:
    posts = graph.get_object(page_id + '/posts?limit=100')
    count = 0
    
    while(True):
        try:
            for post in posts['data']:
                if 'message' in post:
                    posts_text.append(post['message'])
                    posts_label.append(page_ids.index(page_id))
                    count += 1
                    if(count >= fetch_count):
                        break
                        
            if(count >= fetch_count):
                break
                        
            # 找下一100筆資料
            posts = requests.get(posts['paging']['next']).json()
        except KeyError:
            # 後方已無資料
            break
            
posts_count = len(posts_text)

random.seed()
test_idx = random.sample(range(posts_count), test_count)

x_train = []
y_train = []
x_test = []
y_test = []

for i in random.sample(range(posts_count), posts_count):
    if i in test_idx:
        x_test.append(posts_text[i])
        y_test.append(posts_label[i])
    else:
        x_train.append(posts_text[i])
        y_train.append(posts_label[i])
        
print '一共抓取 {} 篇粉絲專頁文章，選擇其中 {} 筆為測試資料，其餘為訓練資料'.format(posts_count, test_count)

一共抓取 10870 篇粉絲專頁文章，選擇其中 100 筆為測試資料，其餘為訓練資料


In [2]:
import jieba
import sys

from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer

# 對 training data 進行 jieba斷詞
corpus = []
for msg in x_train:
    corpus.append(" ".join(jieba.cut(msg, cut_all=False)))
    #print corpus[len(corpus)-1]

# 計算 training data 的 TF-IDF
vectorizer = TfidfVectorizer()
train_tfidf = vectorizer.fit_transform(corpus)

print '訓練資料的TF-IDF計算完畢'

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.403 seconds.
Prefix dict has been built succesfully.


訓練資料的TF-IDF計算完畢


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import numpy

# 對 test data 進行 jieba 斷詞
corpus = []
for msg in x_test:
    corpus.append(" ".join(jieba.cut(msg, cut_all=False)))
    
# 計算 test data 的TF-IDF
tfidf_test = vectorizer.transform(corpus)

print '測試資料的TF-IDF計算完畢'

測試資料的TF-IDF計算完畢


In [4]:
# 測試kNN (k = 10)
clf = KNeighborsClassifier(10)
clf.fit(train_tfidf, y_train)
clf.predict(tfidf_test)
print "kNN (k = 10) 平均準確率: \t", clf.score(tfidf_test, y_test)

kNN (k = 10) 平均準確率: 	0.65


In [5]:
# 測試kNN (k = 50)
clf = KNeighborsClassifier(50)
clf.fit(train_tfidf, y_train)
clf.predict(tfidf_test)
print "kNN (k = 50) 平均準確率: \t", clf.score(tfidf_test, y_test)

kNN (k = 50) 平均準確率: 	0.75


In [6]:
# 測試kNN (k = 100)
clf = KNeighborsClassifier(100)
clf.fit(train_tfidf, y_train)
clf.predict(tfidf_test)
print "kNN (k = 100) 平均準確率: \t", clf.score(tfidf_test, y_test)

kNN (k = 100) 平均準確率: 	0.83


In [7]:
# 測試kNN (k = 500)
clf = KNeighborsClassifier(500)
clf.fit(train_tfidf, y_train)
clf.predict(tfidf_test)
print "kNN (k = 500) 平均準確率: \t", clf.score(tfidf_test, y_test)

kNN (k = 500) 平均準確率: 	0.83


In [8]:
# 測試決策樹演算法
clf = DecisionTreeClassifier()
clf.fit(train_tfidf, y_train)
clf.predict(tfidf_test)
print "決策樹 平均準確率: \t", clf.score(tfidf_test, y_test)

決策樹 平均準確率: 	0.84


In [9]:
# 測試RandomForest
clf = RandomForestClassifier()
clf.fit(train_tfidf, y_train)
clf.predict(tfidf_test)
print "RandomForest 平均準確率: \t", clf.score(tfidf_test, y_test)

RandomForest 平均準確率: 	0.7


In [10]:
# 測試SVM
clf = LinearSVC()
clf.fit(train_tfidf, y_train)
clf.predict(tfidf_test)
print "SVM 平均準確率: \t", clf.score(tfidf_test, y_test)

SVM 平均準確率: 	0.94
